In [14]:
!pip install datasets

In [15]:
from google.colab import drive
import pandas as pd
from transformers import pipeline
from datasets import Dataset
from sklearn.metrics import accuracy_score, classification_report

In [16]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
test = pd.read_csv("/content/drive/My Drive/tcc/data/test.csv")

In [18]:
X_test = test["text"]
y_test = test["label"]

In [19]:
labels = ["esse acórdão foi favorável ao recorrente",
    "o pedido foi julgado procedente",
    "esse acórdão foi desfavorável ao recorrente",
    "o pedido foi julgado improcedente"]

In [20]:
pipe = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli", device = 0)

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [21]:
label_mapping = {
     "esse acórdão foi favorável ao recorrente": 0,
    "o pedido foi julgado procedente": 0,
    "esse acórdão foi desfavorável ao recorrente": 1,
    "o pedido foi julgado improcedente": 1
}

In [22]:
test_dataset = Dataset.from_dict({"text": X_test.tolist()})

In [23]:
def zero_shot_entailment_parallel(batch):
    result = pipe(batch["text"], labels, multi_label=False)
    predicted_label = result["labels"][0]
    return {"final_label": label_mapping[predicted_label]}

In [24]:
classified_dataset = test_dataset.map(zero_shot_entailment_parallel, batched=False)

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

In [25]:
y_pred_entailment = classified_dataset["final_label"]

In [26]:
accuracy_entailment = accuracy_score(y_test, y_pred_entailment)
print(f"Acurácia Zero-Shot: {accuracy_entailment}")

report_entailment = classification_report(y_test, y_pred_entailment)
print("Relatório de Classificação Zero-Shot:\n", report_entailment)

Acurácia Zero-Shot: 0.8076923076923077
Relatório de Classificação Zero-Shot:
               precision    recall  f1-score   support

           0       0.74      0.81      0.77        31
           1       0.86      0.81      0.84        47

    accuracy                           0.81        78
   macro avg       0.80      0.81      0.80        78
weighted avg       0.81      0.81      0.81        78

